In [1]:
import numpy as np
import math

### Динамическое программирование

### 1. Задача о рюкзаке

Представьте что вы воришка пробравшийся в
музыкальный магазин и у вас есть рюкзак
способный унести 4 килограмма.
На выбор есть 3 предмета:
* Гитара (1 кг, 1500 долларов),
*  Tруба (3 кг, 3000 долларов),
*   Аккордеон (3,5 кг, 3000 долларов).
* Какие предметы украсть выгоднее? Ход решения:
* Строка гитара. Пытаемся уложить гитару в
рюкзак. Если ёмкость позволяет кладём
гитару. Потом укладываем Аккордеон, если Аккордеон не влез, поэтому берём самое
выгодное предыдущее решение. Теперь проделаем тот же алгоритм для трубы.

In [7]:
def backpack(weights, values, capacity):
    n = len(values)
    # Создаем матрицу для представления таблицы решений задачи о рюкзаке
    K = [[0 for _ in range(capacity + 1)] for _ in range(n + 1)]
    
    for i in range(n + 1):
        # Заполняем массив K
        for w in range(capacity + 1):
            if i == 0 or w == 0:
                K[i][w] = 0
            elif weights[i-1] <= w:
                # Максимальное значение, это максимум из двух случаев:
                # 1. Предмет не включен.
                # 2. Предмет включен.
                K[i][w] = max(K[i-1][w], values[i-1] + K[i-1][w-weights[i-1]]) 
            else:
                K[i][w] = K[i-1][w] # Если вес предмета больше capacity, предмет не включается.

    return K[n][capacity]

weights = [1*10, 3*10, int(3.5*10)]  
values = [1500, 2000, 3000]
capacity = 40

max_value = backpack(weights, values, capacity)
print(f"Максимум который можно унести: ${max_value}")

Максимум который можно унести: $3500


### 2. Числа Фибоначчи

0, 1, 1, 2, 3, 5, 8, 13, 21, 34, 55, 89, 144, 233, 377, 610, 987, 1597, 2584, 4181, 6765, 10946, 17711, …

Введем переменную dp, в которой будем хранить массив.
При попытке рассчитать новое значение мы будем проверять dp,
на возможное наличие значения в нем.
Только если его не найдем там, будем считать.

In [32]:
def fib(n, dp=None):
    # Инициализация списка dp достаточной длины
    if dp is None:
        dp = [0] * (n + 1)
        
    if n == 0:
        return 0
    if n <= 2:
        dp[n] = 1
        return dp[n]

    # Проверка, вычислено ли значение ранее
    if dp[n] != 0:
        return dp[n]

    # Рекурсивное вычисление значений с мемоизацией
    dp[n] = fib(n-1, dp) + fib(n-2, dp)

    return dp[n]

In [31]:
print(fib(5))
print(fib(10))
print(fib(15))

5
55
610


### 3. Задача про кузнечика

Дана прямая, поделенные на отрезки. В самом
начале этой прямой сидит кузнечик, который
может прыгать только на один или два
отрезка вперед. Необходимо понять, сколько
есть способов допрыгать до n-го отрезка.

Допустим кузнечик может прыгать на K шагов
Hа последний столбик кузнечик может прыгнуть с n-k
столбика: dp[n] = dp[n-1] + dp[n-2] + … + dp[n-k]

In [34]:
def jump(n,k):
    a = [0] * (n+1)
    a[0] = 1
    for i in range(1, n+1):
        r = k
        if i < r:
            r = 1
        a[i] = 0
        for j in range(1, r+1):
            a[i] += a[i-j]
    return a[n]

In [35]:
print(jump(4, 2))

5


In [36]:
print(jump(3, 2))

3


### 4. Кузнечик и монетки

В условия добавляются монетки. На каждой кочке кузнечик может взять монетки, а может потерять.
Теперь ему необходимо добраться до последней кочки собрав максимальное количество монеток

Заводим новый массив jumps, куда мы будем писать наши прыжки
На каждой итерации, в цикле прыжков, мы должны проверять
количество монеток на данном столбике и если оно равно ранее
рассчитанному значению - записываем в массив прыжков:
dp[i] == dp[i - j] + coins[i - 1].
На каждой итерации по i ищем максимальную сумму.

In [47]:
def max_coins(n,k,coins):
    dp = [0] * (n+1)
    for i in range(1, n+1):
        for j in range(1, min(k,i)+1):
            dp[i] = max(dp[i],dp[i-j] + coins[i-1])
    max_coins_col = dp[n]

    jumps = []
    i = n
    while i > 0:
        for j in range(min(k,i),0,-1):
            if dp[i] == dp[i-j] + coins[i-1]:
                jumps.append(i)
                i -= j
                break
                
    jumps.reverse()
    return max_coins_col, len(jumps),jumps

In [48]:
n = 6
k = 2
coins = [0, 4, 15, 9, -7, 0]
result = max_coins(n, k, coins)
print("Максимальное количество монет:",
result[0])
print("Количество прыжков:", result[1])
print("Последовательность прыжков:", result[2])

Максимальное количество монет: 28
Количество прыжков: 4
Последовательность прыжков: [2, 3, 4, 6]


### 5. Черепашка и монетки

Теперь будем собирать монетки в
двухмерном пространстве. Наша черепашка может двигать только
вправо и вниз. Нужно собрать как можно больше монет.
Движение по первым горизонтали и вертикали:
* dp[0][k] = dp[0][k - 1] + COINS[0][k]
* dp[k][0] = dp[k - 1][0] + COINS[k][0]
* dp[0][k] = dp[0][k - 1] + COINS[0][k]
* dp[k][0] = dp[k - 1][0] + COINS[k][0]
* dp[i][j] = max(dp[i - 1][j], dp[i][j - 1]) + COINS[i][j]

In [59]:
M = 6; N = 4
COINS = [
    [0, 1, 1, 1, 1, 1],
    [0, 0, 0, 0, 0, 1],
    [0, 40, 70, 0, 0, 1],
    [100, 0, 0, 0, 0, 1]
]
dp = [[None] * M for i in range(N)]
prev = [[None] * M for i in range(N)]
for i in range(N):
    for j in range(M):
        if i == 0 and j == 0:
            dp[0][0] = COINS[0][0]
            prev[0][0] = -1 # предыдущей клетки нет
        elif i == 0:
            dp[0][j] = dp[0][j - 1] + COINS[0][j]
            prev[0][j] = 0 # слева пришли
        elif j == 0:
            dp[i][0] = dp[i - 1][0] + COINS[i][0]
            prev[i][0] = 1 # сверху пришли
        else:
            dp[i][j] = max(dp[i - 1][j], dp[i][j - 1]) + COINS[i][j]
            if dp[i - 1][j] > dp[i][j - 1]:
                prev[i][j] = 1
            else:
                prev[i][j] = 0
    print(prev[i])

i, j = N - 1, M - 1
answer = []
answer_directions = []
while i > 0 or j > 0:
    if i != 0 and (j == 0 or dp[i - 1][j] > dp[i][j - 1]):
        i -= 1
        answer_directions.append('DOWN')
    else:
        j -= 1
        answer_directions.append('RIGHT')
        answer.append((i, j))
print(answer[::-1]) # reverse
print(answer_directions[::-1]) # reverse

[-1, 0, 0, 0, 0, 0]
[1, 1, 1, 1, 1, 1]
[1, 1, 0, 0, 0, 0]
[1, 0, 1, 0, 0, 1]
[(0, 0), (2, 1), (2, 2), (2, 3), (2, 4)]
['RIGHT', 'DOWN', 'DOWN', 'RIGHT', 'RIGHT', 'RIGHT', 'RIGHT', 'DOWN']
